In [2]:
import xarray as xr
import matplotlib.pyplot as plt

In [18]:

ds = xr.open_dataset("../Example_3007.nc")

sal1d = ds.salinity.isel(mesh2d_nFaces=222, mesh2d_nLayers=20)
salOut1d = ds.output_salinity.isel(mesh2d_nFaces=222, mesh2d_nLayers=20)

print(sal1d.values)
sal1d.plot()

ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'scipy']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html